In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import json
import torch
from tqdm import tqdm

model_name = "qwen_0.6B_full_bs_1_grac_8_lr_5e-5_epoch_3_max_5120_t2"  # 你的保存路径
model_dir = f"./checkpoints/{model_name}"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_dir)
#base_model = AutoModelForCausalLM.from_pretrained(
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

with open("./data/test_processed_q1.json") as f:
    test_data = json.load(f)
result = []

/home/yunpeng/anaconda3/envs/anqi-lab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for data in tqdm(test_data[:10]):
    # prepare the model input
    prompt = data["prompt"]
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    print(content)

 10%|█         | 1/10 [00:02<00:24,  2.70s/it]

[{'name': '陈某', 'judgment': [{'standard_accusation': '抢夺罪', 'imprisonment': 12}]}, {'name': '贺某某', 'judgment': [{'standard_accusation': '抢夺罪', 'imprisonment': 10}]}]


 20%|██        | 2/10 [00:04<00:19,  2.43s/it]

[{'name': '马某某', 'judgment': [{'standard_accusation': '寻衅滋事罪', 'imprisonment': 0}]}, {'name': '孙某某', 'judgment': [{'standard_accusation': '寻衅滋事罪', 'imprisonment': 0}]}]


 30%|███       | 3/10 [00:08<00:19,  2.75s/it]

...

[{'name': '曾某某', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 12}]}, {'name': '苏某某', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 30}]}, {'name': '张某某', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 24}]}, {'name': '齐作国如', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 12}]}, {'name': '殷某某', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 12}]}, {'name': '殷某某', 'judgment': [{'standard_accusation': '盗窃罪', 'imprisonment': 24}]}]


 90%|█████████ | 9/10 [00:22<00:03,  3.23s/it]

[{'name': '杨某某', 'judgment': [{'standard_accusation': '寻衅滋事罪', 'imprisonment': 14}]}, {'name': '肖某A', 'judgment': [{'standard_accusation': '寻衅滋事罪', 'imprisonment': 14}]}, {'name': '谢某某', 'judgment': [{'standard_accusation': '寻衅滋事罪', 'imprisonment': 12}]}]


100%|██████████| 10/10 [00:27<00:00,  2.72s/it]

[{'name': '王某', 'judgment': [{'standard_accusation': '虚开发票罪', 'imprisonment': 24}]}, {'name': '钱某某', 'judgment': [{'standard_accusation': '虚开发票罪', 'imprisonment': 24}]}, {'name': '张某某', 'judgment': [{'standard_accusation': '虚开发票罪', 'imprisonment': 36}]}, {'name': '纪某某', 'judgment': [{'standard_accusation': '虚开发票罪', 'imprisonment': 24}]}]


In [1]:
import json

with open("results/qwen_0.6B_full_bs_1_grac_8_lr_5e-5_epoch_3_max_5120_t2_refine_prompt/results.json", "r") as f:
    results = json.load(f)
with open("data/test_processed_q2.json", "r") as f:
    test_data = json.load(f)

In [4]:
print(results[0])
print(test_data[0]["label"])

[{'name': '陈某', 'judgment': [{'standard_accusation': '抢夺罪', 'imprisonment': 12}]}, {'name': '贺某某', 'judgment': [{'standard_accusation': '抢夺罪', 'imprisonment': 12}]}]
[{'name': '陈某', 'judgment': [{'standard_accusation': '', 'imprisonment': ''}]}, {'name': '贺某某', 'judgment': [{'standard_accusation': '', 'imprisonment': ''}]}]


In [10]:
for i in range(1000):
    res = results[i]
    data = eval(test_data[i]["label"])
    if len(res) != len(data):
        print(f"People Error at index {i}: {res} vs {data['label']}")

In [11]:
with open("data/train_processed_q2.json", "r") as f:
    train_data = json.load(f)

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
        "model_zoo/Qwen3-0.6B",
        trust_remote_code=True,
        use_fast=True
    )

item = train_data[0]

prompt = item["prompt"]
messages = [{"role": "user", "content": prompt}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
inputs = tokenizer([text], return_tensors="pt")

/home/yunpeng/anaconda3/envs/anqi-lab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
inputs['input_ids'].shape

torch.Size([1, 2125])

In [19]:
text

"<|im_start|>user\n【案件事实】\n某河市源汇区人民检察院指控，2011年至2013年期间，被告单位某甲公司为了排除竞争、更好地在某河开展业务，分多次向某河市建设工程招标投标管理办公室（以下简称某河市招标办）行贿人民币79万元和一辆价值15.73万元的别克英朗汽车。1、2011年1月，某甲公司副总经理靳某某要求某甲公司驻某河办事处负责人赵某某为某河市招标办解决10万元经费。2011年1月25日，河南某甲公司经其财务副总经理吴某同意后给赵某某账户（账号为52409425600025894）转账10万元，赵某某于2011年1月25日将10万元交给某河招标办会计冯某某。2、2012年3月31日，某甲公司经其总经理靳某某和财务副总经理吴某同意后，为某河市招标办解决5万元经费。3、2012年4月13日，某甲公司经其总经理靳某某和财务副总经理吴某同意后，为某河市招标办购买价值15.73万元别克牌英朗汽车一部，杜某某于2012年4月24日将该车过户到其朋友赵某甲名下。4、2012年12月4日，某甲公司驻某河办事处负责人刘某某经某甲公司总经理靳某某、财务副总经理吴某同意后，为某河市招标办解决2万元经费。5、2013年1月，某河市招标办主任杜某某要求某甲公司为招标办解决经费，某甲公司总经理靳某某和公司副总经理吴某同意后，靳某某向某甲公司支取32万元现金。杜某某在2013年1月到某甲公司靳某某办公室拿走现金32万元。6、2013年9月，某河市招标办主任杜某某提出让某甲公司驻某河办事处负责人刘某某为招标办解决经费，刘某某向某甲公司总经理靳某某汇报，经某甲公司总经理靳某某和财务副总经理吴某同意后，某甲公司会计李某甲为某河招标办会计冯某某提供的账户（户名殷某，账号000368660400010）转款30万元。案发后，非法所得予以追回。公诉机关提供情况说明、营业执照、银行汇款凭证、账本、证人赵某某等人证言、被告人靳某某等人的供述，认为被告单位某甲公司、被告人靳某某、刘某某、吴某等的行为均已构成x罪，被告人靳某某、刘某某、吴某有自首情节，请求本院依法判处。被告单位某甲公司辩称其行为不构成犯罪。其辩护人辩称，第二、四起指控证据不足，公司未谋取不正当利益，相关钱款是招标办索要，某甲公司不构成x罪。被告人靳某某对起诉指控无异议，当庭自愿认罪。其辩护人提出，靳某某不是决策者，是执行者，

In [42]:
label = train_data[0]["label"]

In [45]:
label

"[{'name': '靳某某', 'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]}, {'name': '刘某某', 'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]}, {'name': '吴某', 'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]}]"

In [43]:
import random
l = eval(label)
l
# label = str([random.shuffle(people['judgment']) for people in label])

[{'name': '靳某某',
  'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]},
 {'name': '刘某某',
  'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]},
 {'name': '吴某',
  'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]}]

In [37]:
[random.shuffle(people['judgment']) for people in l]

[None, None, None]

In [38]:
l[0]

{'name': '靳某某',
 'judgment': [{'standard_accusation': '对单位行贿罪', 'imprisonment': 0}]}

In [41]:
random.shuffle(l[0]['judgment'])

In [44]:
l = eval(label)
for people in l:
    random.shuffle(people['judgment'])
label = str(l)

In [46]:
import json

with open("./data/articles.json", "r") as f:
    data = json.load(f)

with open("./data/train_processed_q1.json", "r") as f:
    train_data = json.load(f)

article_data = []
for item in data.values():
    title, content = item.split('条：')
    title = title + '条'
    article_data.append({
        "prompt": f"请默写{title}",
        "label": content
    })

In [47]:
article_data[0]

{'prompt': '请默写《中华人民共和国刑法》第一百一十四条',
 'label': '放火、决水、爆炸以及投放毒害性、放射性、传染病病原体等物质或者以其他危险方法危害公共安全，尚未造成严重后果的，处三年以上十年以下有期徒刑。'}

In [54]:
import json
from tqdm import tqdm
from transformers import AutoTokenizer
dataset = "train"
with open(f"./data/{dataset}.jsonl") as f:
    test_data = [json.loads(line) for line in f]


In [55]:
test_data[0]

{'fact': '某河市源汇区人民检察院指控，2011年至2013年期间，被告单位某甲公司为了排除竞争、更好地在某河开展业务，分多次向某河市建设工程招标投标管理办公室（以下简称某河市招标办）行贿人民币79万元和一辆价值15.73万元的别克英朗汽车。1、2011年1月，某甲公司副总经理靳某某要求某甲公司驻某河办事处负责人赵某某为某河市招标办解决10万元经费。2011年1月25日，河南某甲公司经其财务副总经理吴某同意后给赵某某账户（账号为52409425600025894）转账10万元，赵某某于2011年1月25日将10万元交给某河招标办会计冯某某。2、2012年3月31日，某甲公司经其总经理靳某某和财务副总经理吴某同意后，为某河市招标办解决5万元经费。3、2012年4月13日，某甲公司经其总经理靳某某和财务副总经理吴某同意后，为某河市招标办购买价值15.73万元别克牌英朗汽车一部，杜某某于2012年4月24日将该车过户到其朋友赵某甲名下。4、2012年12月4日，某甲公司驻某河办事处负责人刘某某经某甲公司总经理靳某某、财务副总经理吴某同意后，为某河市招标办解决2万元经费。5、2013年1月，某河市招标办主任杜某某要求某甲公司为招标办解决经费，某甲公司总经理靳某某和公司副总经理吴某同意后，靳某某向某甲公司支取32万元现金。杜某某在2013年1月到某甲公司靳某某办公室拿走现金32万元。6、2013年9月，某河市招标办主任杜某某提出让某甲公司驻某河办事处负责人刘某某为招标办解决经费，刘某某向某甲公司总经理靳某某汇报，经某甲公司总经理靳某某和财务副总经理吴某同意后，某甲公司会计李某甲为某河招标办会计冯某某提供的账户（户名殷某，账号000368660400010）转款30万元。案发后，非法所得予以追回。公诉机关提供情况说明、营业执照、银行汇款凭证、账本、证人赵某某等人证言、被告人靳某某等人的供述，认为被告单位某甲公司、被告人靳某某、刘某某、吴某等的行为均已构成x罪，被告人靳某某、刘某某、吴某有自首情节，请求本院依法判处。被告单位某甲公司辩称其行为不构成犯罪。其辩护人辩称，第二、四起指控证据不足，公司未谋取不正当利益，相关钱款是招标办索要，某甲公司不构成x罪。被告人靳某某对起诉指控无异议，当庭自愿认罪。其辩护人提出，靳某某不是决策者，是执行者，其有法定减轻情节，建议对其免予刑事

In [63]:
model_dir = "model_zoo/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
result = []
for i in tqdm(range(len(test_data))):
    item = test_data[i]
    fact = item["fact"]
    prompt = f"""
假设你是一个法律专家。请你将以下犯罪事实压缩到**2000**字左右。要求：
1）保留可以用来断定每个犯罪嫌疑人**罪名**和**量刑**的关键信息；
2）不要对**嫌疑犯**的人名进行任何程度的改动与删减；
3）**可以**删去与判定罪行无关的证据收集、调查过程等内容与人物；
4）不需要给出“好的”之类的答复，而是**直接输出**整理后的事实。
5）所有输出严禁换行，严禁使用Markdown语法
6）所有内容输出在同一行，输出纯文本。
以下是犯罪事实。
{fact}
"""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    inputs = tokenizer([text], return_tensors="pt")
    if inputs["input_ids"].shape[1] > 4608:
        result.append(i)


100%|██████████| 10000/10000 [00:30<00:00, 332.14it/s]


In [64]:
len(result)

484

In [73]:
with open("./data/long_data_index.json", "w") as f:
    json.dump(new_res, f, ensure_ascii=False, indent=2)

In [71]:
new_res = []

for res in result:
    if (res+2)%8 != 0:
        new_res.append(res)
    else:
        print(res)

302
414
446
550
710
726
798
814
1030
1214
1534
1630
1726
1750
2214
2230
2430
2462
2670
2950
2974
2998
3078
3206
3502
3726
3742
3846
3862
3982
4006
4014
4422
4718
4862
5166
5302
5334
5358
5558
5574
5798
5830
5870
5926
6022
6078
6142
6382
6398
6598
6766
6854
6886
7134
7414
7430
7534
7878
8054
8086
8142
8166
8214
8222
8486
8702
8974
9414
9454
9646
9670
9942
9950
9990


In [72]:
test_data[302]

{'fact': '某市某区人民检察院指控：（一）被告人王某C滥用职权2017年4月，某市某区某镇成立两河区域征迁指挥领导小组（以下简称征迁领导小组），负责某镇畜牧养殖小区（以下简称养殖小区）征迁。2017年7月，某市某区价格认证中心（以下简称区价格认证中心）接受某镇政府委托，对养殖小区征迁进行价格认证，李某5（另处）具体负责。征迁方案对2008年前和2016年前的住房、生产用房给予不同补偿和奖励，2017年以后建筑不予补偿。在征迁过程中，征迁领导小组组长、镇长王某2、副组长汪某及某村原党总支部书记被告人王某A、区价格认证中心李某5等人，对不在2016年航拍图上建筑补偿问题向被告人王某C汇报。王某C违反征迁方案，擅自同意以2017年3月航拍为依据，进行征迁补偿。2017年10月，某市某区审计局（以下简称区审计局）对补偿金额超过1000万元的补偿协议进行审计，发现某市爱农畜牧有限责任公司（以下简称爱农公司）拆分的郭某、刘某1养殖场及田某2养殖场征迁补偿面积与航拍图差别过大，不能按照征迁协议支付征迁补偿款。被告人王某C得知消息后，不履行监督管理职责，进行认真核实查明事实，反而通过与区审计局、区价格认证中心等相关承办人协调，并向区政府领导不实汇报，促使征迁资金发放，致使养殖小区征迁资金流失1658万余元。（二）被告人王某C受贿265.18万元，被告人王某A伙同李某5共同受贿200万元（1）被告人王某C收受爱农公司杨某2人民币200万元爱农公司、某市新啟养殖有限责任公司（以下简称新啟公司）、某市秋农养殖专业合作社（以下简称秋农合作社）实际控制人杨某2，为在征收过程中得到被告人王某C的帮助，允诺给予王某C回报。2017年11月，审计发现杨某2养殖场征收补偿协议认定面积与航拍图差值过大，补偿款不能拨付。杨某2遂找到王某C请求帮助。王某C明知杨某2补偿内容与实际情况不符，仍在补偿资金拨付上给予帮助，帮助杨某2了解差异原因，并协调审计、评估部门，向区领导虚假汇报，认可评估结论，安排某镇财政所邱某与杨某2到某地请相关领导签署拨款手续。2017年12月下旬，杨某2征迁补偿款顺利拨付。2017年12月底至2018年1月6日，王某C收受杨某2人民币200万元。2018年4月前后，省委巡视组巡视发现被告人王某C涉嫌受贿利益输送等问题。后王某C为掩盖其收受杨某2人民币200万元事实，多次与杨某

In [74]:
with open("./data/articles.json", "r") as f:
    data = json.load(f)

In [79]:
article_data = []

for item in data.values():
    # 找到所有“的，”的位置
    parts = item.split("的，")
    if len(parts) < 2:
        continue  # 如果没有“的，”，就跳过

    # 构造多个训练样本，每次往后多一个“的，”片段
    for i in range(1, len(parts)):
        title = "的，".join(parts[:i]) + "的，"
        content = "的，".join(parts[i:])
        article_data.append({
            "prompt": f"""请补全以下法条：
{title}""",
            "label": content
        })

In [81]:
with open("./data/articles_cloze.json", "w") as f:
    json.dump(article_data, f, ensure_ascii=False, indent=2)

In [82]:
import re

def convert_years_to_months(text):
    def year_str_to_int(s):
        """将中文数字或阿拉伯数字转为整数"""
        cn2num = {'一':1,'二':2,'三':3,'四':4,'五':5,'六':6,'七':7,'八':8,'九':9,'十':10}
        if s.isdigit():
            return int(s)
        elif s == '十':
            return 10
        elif len(s) == 2 and s[0] == '十':  # 十三 => 13
            return 10 + cn2num.get(s[1], 0)
        elif len(s) == 2 and s[1] == '十':  # 三十 => 30
            return cn2num.get(s[0], 0) * 10
        elif len(s) == 3 and s[1] == '十':  # 二十三 => 23
            return cn2num.get(s[0], 0) * 10 + cn2num.get(s[2], 0)
        return cn2num.get(s, 0)

    # 匹配：“三年以下”“五年以上”“五年以上十年以下”等
    pattern = re.compile(r"(?P<start>[\d一二三四五六七八九十]+)年(?P<type>以上|以下)(?:(?P<end>[\d一二三四五六七八九十]+)年以下)?")

    def replace(match):
        start = year_str_to_int(match.group('start')) * 12
        _type = match.group('type')
        end_raw = match.group('end')

        if end_raw:
            end = year_str_to_int(end_raw) * 12
            return f"{start}个月{_type}{end}个月以下"
        else:
            return f"{start}个月{_type}"

    return pattern.sub(replace, text)

# 应用于所有法条
converted_data = {}
for k, v in data.items():
    converted_data[k] = convert_years_to_months(v)

In [86]:
list(converted_data.values())[0]

'《中华人民共和国刑法》第一百一十四条：放火、决水、爆炸以及投放毒害性、放射性、传染病病原体等物质或者以其他危险方法危害公共安全，尚未造成严重后果的，处36个月以上120个月以下有期徒刑。'

In [87]:
article_data = []

for item in converted_data.values():
    # 找到所有“的，”的位置
    parts = item.split("的，")
    if len(parts) < 2:
        continue  # 如果没有“的，”，就跳过

    # 构造多个训练样本，每次往后多一个“的，”片段
    for i in range(1, len(parts)):
        title = "的，".join(parts[:i]) + "的，"
        content = "的，".join(parts[i:])
        article_data.append({
            "prompt": f"""请补全以下法条：
{title}""",
            "label": content
        })

In [88]:
with open("./data/articles_month_cloze.json", "w") as f:
    json.dump(article_data, f, ensure_ascii=False, indent=2)

In [89]:
len(article_data)

938

In [ ]:
with open(f"./data/train_processed_q2.json", "r") as f:
    train_data = json.load(f)
train_data.extend(article_data)
train_data.extend(article_data)
with open(f"./data/train_processed_q2_month_cloze_2x.json", "w") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

FileNotFoundError: [Errno 2] No such file or directory: 'train_processed_q2.json'